### 3D City Models from Volunteered Public Data

**This notebook will produce an interactive 3D Building Model visualization** *- via [pyrosm](https://pyrosm.readthedocs.io/en/latest/) with [pydeck](https://deckgl.readthedocs.io/en/latest/) -* **which you can navigate and query.** It differs from [InteractiveOnly.ipynb](https://github.com/AdrianKriger/osm_LoD1_3DCityModel/blob/main/districts/interactiveOnly.ipynb) and [InteractiveOnlyMany.ipynb](https://github.com/AdrianKriger/osm_LoD1_3DCityModel/blob/main/districts/extra/interactiveOnlyMany.ipynb) through harvesting many areas after trimming a `country.osm.pbf` with a `.poly`.

In [30]:
#load the magic

%matplotlib inline
import os
import subprocess
from pathlib import Path
import requests

from pyrosm import OSM, data
from pyrosm import get_data

import pyproj

import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Polygon, shape, mapping
#shapely.speedups.disable()
from shapely.ops import transform

import json
import geojson

from osgeo import ogr

from openlocationcode import openlocationcode as olc

import pydeck as pdk

**Harvest [OpenStreetMap](https://en.wikipedia.org/wiki/OpenStreetMap)** - interogate an osm.pbf (["Protocolbuffer Binary Format"](https://wiki.openstreetmap.org/wiki/PBF_Format)) with [pyrosm]() from within Jupyter and convert to .geojson.

<div class="alert alert-block alert-info"><b></b> 
    
While the building data has already been harvested and saved to .geojson through executing [osm3DMainDistricts.py](https://github.com/AdrianKriger/osm_LoD1_3DCityModel/blob/main/districts/osm3DMainDistricts.py); we start with a blank slate.
</div>

In [31]:
# update=True to download a fresh osm.pbf
fp = get_data("South Africa", update=False, directory="data")

In [32]:
# set a few path's
path = os.getcwd()
osm_convert_path = os.path.join('osmconvert64')                            #-- osmconvert.exe
in_path = fp                                                               #-- country.osm.pbf as input
poly_path = os.path.join(path, Path('data', 'tshwane_gauteng.poly'))       #-- .poly in data-folder
out_path = os.path.join(path, Path('data', 'Tshwane_gauteng_extract.pbf')) #-- new.osm.pbf to data-folder
# -- run osmconvert64
subprocess.call('{} {} -B={} -o={}'.format(osm_convert_path, in_path, poly_path, out_path))
# -- read the new.osm.pbf
fp = out_path

In [33]:
osm = OSM(fp)

**One `boundary=adminstrative` or comma seperated**

In [34]:
area = "Tshwane Ward 81,Tshwane Ward 92,Tshwane Ward 56,Tshwane Ward 82"

In [35]:
shapes = area.split(",") 
if len(shapes) == 1:
        aoi = osm.get_boundaries(name=shapes[0])
              
if len(shapes) > 1:
    df = pd.DataFrame()
        
    for i in shapes:
        aoi = osm.get_boundaries(name=i)
        df = df.append(aoi)
        
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    aoi = gdf.dissolve()

In [36]:
# Get the shapely geometry from GeoDataFrame
bbox_geom = aoi['geometry'].values[0]
# Initiliaze with bounding box
osm = OSM(fp, bounding_box=bbox_geom)
# Retrieve buildings
ts = osm.get_buildings(extra_attributes=["addr:suburb"])

In [37]:
# basic cleaning to harvest buildings with level tags only
ts.dropna(subset=['building:levels'], inplace= True)
ts['building:levels'] = pd.to_numeric(ts['building:levels'], downcast='integer')
ts['building:levels'] = ts['building:levels'].astype(int)
ts = ts[ts['building:levels'] != 0]
ts['building:levels'] = ts['building:levels'].astype(float)

In [38]:
#what do we have?
ts.head(2)

,addr:city,addr:country,addr:housenumber,addr:housename,addr:postcode,addr:street,email,name,opening_hours,operator,phone,ref,url,website,building,amenity,building:flats,building:levels,building:material,building:max_level,building:min_level,building:use,craft,height,internet_access,landuse,office,shop,source,start_date,wikipedia,addr:suburb,id,timestamp,version,geometry,tags,osm_type,changeset
5,Pretoria,None,357,None,0002,Elandspoort,None,Musaion,None,None,None,None,None,None,university,None,None,3.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,103768391,1604421986,6,"POLYGON ((28.22867 -25.75486, 28.22850 -25.754...","{""description"":""Department of Music"",""field"":""...",way,NaN
6,Pretoria,None,357,None,0002,Elandspoort,None,Amphitheatre,None,None,None,None,None,None,university,None,None,3.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,103768396,1604421748,7,"POLYGON ((28.22880 -25.75436, 28.22888 -25.754...","{""description"":""Department of Music"",""field"":""...",way,NaN


<div class="alert alert-block alert-warning"><b>Calculate building height:</b> 

We assume a building level is 2.8 meters high and add another 1.3 meters (to account for the roof) and create a new attribute height.</div>

In [39]:
# I'm not knowledgable enough to go directly from a .gdf to pydeck and have to go through .geojson

storeyheight = 2.8
    #-- iterate through the list of buildings and create GeoJSON 
    # features rich in attributes
footprints = {
    "type": "FeatureCollection",
    "features": []
    }
columns = ts.columns   
for i, row in ts.iterrows():
    f = {
    "type" : "Feature"
    }
    f["properties"] = {}      
        #-- store all OSM attributes and prefix them with osm_ 
    f["properties"]["osm_id"] = row.id
    adr = []
            #-- transform the OSM address to string prefix with osm_
    if 'addr:flats' in columns and row['addr:flats'] != None:
        adr.append(row['addr:flats'])
    if 'addr:housenumber' in columns and row['addr:housenumber'] != None:
        adr.append(row['addr:housenumber'])
    if 'addr:housename' in columns and row['addr:housename'] != None:
        adr.append(row['addr:housename'])
    if 'addr:street' in columns and row['addr:street'] != None:
        adr.append(row['addr:street'])
    if 'addr:suburb' in columns and row['addr:suburb'] != None:
        adr.append(row['addr:suburb'])
    if 'addr:postcode' in columns and row['addr:postcode'] != None:
        adr.append(row['addr:postcode'])
    if 'addr:city' in columns and row['addr:city'] != None:
        adr.append(row['addr:city'])
    if 'addr:province' in columns and row['addr:province'] != None:
        adr.append(row['addr:province'])
    
    #f["properties"]["osm_address"] = " ".join(adr)
    f["properties"]["osm_address"] = ' '.join(str(v) for v in adr)
    
    # harvest some tags ~ we could harvest all but lets do less
    if 'building' in columns and row['building'] != None:
        f["properties"]["osm_building"] = row['building']
    if 'amenity' in columns and row['amenity'] != None:
        f["properties"]["osm_amenity"] = row['amenity']
    if 'start_date' in columns and row['start_date'] != None:
        f["properties"]["osm_start_date"] = row['start_date']
    if 'shop' in columns and row['shop'] != None:
        f["properties"]["osm_shop"] = row['shop']
    if 'building:levels' in columns and row['building:levels'] != None:
        f["properties"]["osm_building:levels"] = row['building:levels']
    if 'name' in columns and row['name'] != None:
        f["properties"]["osm_name"] = row['name']
    if 'school' in columns and row['school'] != None:
        f["properties"]["osm_school"] = row['school']
          
    osm_shape = row["geometry"] # shape(row["geometry"][0])
        #-- a few buildings are not polygons, rather linestrings. This converts them to polygons
        #-- rare, but if not done it breaks the code later
    if osm_shape.type == 'LineString':
        osm_shape = Polygon(osm_shape)
        #-- and multipolygons must be accounted for
    elif osm_shape.type == 'MultiPolygon':
            #osm_shape = Polygon(osm_shape[0])
        for poly in osm_shape:
            osm_shape = Polygon(poly)#[0])
            
    p = osm_shape.representative_point()
    f["properties"]["plus_code"] = olc.encode(p.y, p.x, 11)
    
    f["geometry"] = mapping(osm_shape)
        #-- finally calculate the height and store it as an attribute

    f["properties"]['building_height'] = round(float(row['building:levels']) * storeyheight + 1.3, 2) 
    footprints['features'].append(f)

In [42]:
# have a look at a random feature
footprints['features'][2]

{'type': 'Feature',
 'properties': {'osm_id': 103768400,
  'osm_address': '357 Elandspoort 0002 Pretoria',
  'osm_building': 'university',
  'osm_building:levels': 1.0,
  'osm_name': 'Music sciences',
  'plus_code': '5G6C66VH+XGR',
  'building_height': 4.1},
 'geometry': {'type': 'Polygon',
  'coordinates': (((28.2282848, -25.7550405),
    (28.228601, -25.7552363),
    (28.2286769, -25.7551367),
    (28.2284889, -25.7550202),
    (28.2285326, -25.754963),
    (28.2287975, -25.755127),
    (28.2288148, -25.7551044),
    (28.2289068, -25.7551613),
    (28.228987, -25.7550562),
    (28.2288772, -25.7549882),
    (28.2286737, -25.7548622),
    (28.228502, -25.7547559),
    (28.2282848, -25.7550405)),)}}

In [43]:
#-- store the data as GeoJSON
with open('./data/fp_j.geojson', 'w') as outfile:
    json.dump(footprints, outfile)

In [44]:
data = './data/fp_j.geojson'
json = pd.read_json(data)#data)
build_df = pd.DataFrame()

# Parse the geometry out to Pandas
build_df["coordinates"] = json["features"].apply(lambda row: row["geometry"]["coordinates"])
build_df["height"] = round(json["features"].apply(lambda row: row["properties"]["building_height"]), 1)
build_df["plus_codes"] = json["features"].apply(lambda row: row["properties"]["plus_code"])

***~ In order to make the most of the semantic data we need to extract the `osm_tags` from the dictionary: and add it as `tooltips` to the visualization.***

<div class="alert alert-block alert-success"><b>Building Stock:</b> To differentiate a school, housing, retail, healthcare and community focused facilities (library, municipal office, community centre) we color the buildings - we harvest the osm tags [building type] directly.</div>

In [45]:
#we want to display data so extract values from the dictionary 
build_df["address"] = json["features"].apply(lambda row: row["properties"]["osm_address"])
build_df["building:levels"] = json["features"].apply(lambda row: row["properties"]["osm_building:levels"])
build_df["building"] = json["features"].apply(lambda row: row["properties"]["osm_building"])

In [46]:
# have a look at the building types
build_df['building'].unique()

array(['university', 'apartments', 'yes', 'school', 'retail',
       'commercial', 'library', 'Campus', 'office', 'residential',
       'house', 'hospital', 'detached', 'historical', 'chapel', 'garage',
       'hut', 'service', 'roof', 'dormitory', 'college', 'hotel',
       'carport', 'shed', 'pavilion', 'terrace', 'garages', 'law_office',
       'industrial', 'educational', 'parking', 'construction'],
      dtype=object)

In [47]:
#-- colour the building stock based on building:type

## while we can color with a built-in pydeck function
#color_lookup = pdk.data_utils.assign_random_colors(build_df['building'])
 # Assign a color
#build_df['color'] = build_df.apply(lambda row: color_lookup.get(row['building']), axis=1)

## we define specific colors
def color(bld):
    if bld == 'house':# or bld == 'yes':
        return [255, 255, 204]
    if bld == 'office' or bld == 'commercial':
        return [253, 141, 60]
    if bld == 'school':
        return [128, 0, 38]
    if bld == 'clinic' or bld == 'doctors':
        return [89, 182, 178]
    if bld == 'community_centre' or bld == 'service' or bld ==  'post_office' \
    or bld ==  'townhall':
        return [181, 182, 89]
    if bld == 'library':
        return [193, 255, 193]
    if bld == 'retail' or bld == 'restaurant' or bld == 'mall' or bld == 'supermarket':
        return [139, 117, 0]
    if bld == 'place_of_worship':
        return [225, 225, 51]
    else:
        return [255, 255, 204]

build_df["color"] = build_df['building'].apply(lambda x: color(x))

In [48]:
#  what do we have
build_df.head(2)

,coordinates,height,plus_codes,address,building:levels,building,color
0,"[[[28.2286737, -25.7548622], [28.228502, -25.7...",9.7,5G6C66WH+3GR,357 Elandspoort 0002 Pretoria,3.0,university,"[255, 255, 204]"
1,"[[[28.228799, -25.7543621], [28.2288787, -25.7...",9.7,5G6C66WH+6J3,357 Elandspoort 0002 Pretoria,3.0,university,"[255, 255, 204]"


**[pydeck](https://deckgl.readthedocs.io/en/latest/) needs an area and a center**, harvest from the [pyrosm](https://pyrosm.readthedocs.io/en/latest/basics.html#read-boundaries) boundary.

In [ ]:
bounds = aoi.geometry.bounds
x = aoi.centroid.x
y = aoi.centroid.y

bbox = [(bounds.minx, bounds.miny), (bounds.minx, bounds.maxy), 
        (bounds.maxx, bounds.maxy), (bounds.maxx, bounds.miny)]

*I want additional visual effects to show the potential and power of **3D City Models**; namely: parks. We get this from [pyrosm](https://pyrosm.readthedocs.io/en/latest/) as well.* ~ we use the same `osm.pdf`.

In [50]:
# the parks
my_filter = {'leisure':['park']}            
park = osm.get_data_by_custom_criteria(custom_filter=my_filter)

In [51]:
## ~ (x, y) - bl, tl, tr, br  ~~ or ~~ sw, nw, ne, se
#area = [[[18.4377, -33.9307], [18.4377, -33.9283], [18.4418, -33.9283], [18.4418, -33.9307]]]
area = [[[bbox[0][0][0], bbox[0][1][0]], [bbox[1][0][0], bbox[1][1][0]], 
         [bbox[2][0][0], bbox[2][1][0]], [bbox[3][0][0], bbox[3][1][0]]]]

## ~ (y, x)
view_state = pdk.ViewState(latitude=y[0], longitude=x[0], zoom=14, max_zoom=19, pitch=72, 
                                   bearing=80)

land = pdk.Layer(
    "PolygonLayer",
    area,
    stroked=False,
    # processes the data as a flat longitude-latitude pair
    get_polygon="-",
    get_fill_color=[0, 0, 0, 1],
    #material = True,
    #shadowEnabled = True
)
building_layer = pdk.Layer(
    "PolygonLayer",
    build_df,
    #'GeoJsonLayer',
    #data=ts.geometry.__geo_interface__,
    #id="geojson",
    opacity=0.3,
    stroked=False,
    get_polygon="coordinates",
    get_elevation="height",
    filled=True,
    extruded=True,
    wireframe=False,
    get_fill_color="color", #255, 255, 255
    #get_fill_color="fill_color",
    get_line_color="color",#"fill_color",#[255, 255, 255],
    #material = True, 
    #shadowEnabled = True, 
    auto_highlight=True,
    pickable=True,
)

p_layer = pdk.Layer(
    #"PolygonLayer",
    'GeoJsonLayer',
    data=park.geometry.__geo_interface__,
    #id="geojson",
    opacity=0.3,
    stroked=False,
    filled=True,
    extruded=False,
    wireframe=False,
    get_fill_color="[0,128,0]", #255, 255, 255
    get_line_color="[0,128,0]",#"fill_color",#
    auto_highlight=True,
    pickable=False,
)

tooltip = {"html": "<b>Levels:</b> {building:levels} <br/> <b>Address:</b> {address}\
<br/> <b>Plus Code:</b> {plus_codes} <br/> <b>Building Type:</b> {building}"}

r = pdk.Deck(layers=[land, building_layer, p_layer],# b_layer, use_layer],#, water_layer, ], #
             #views=[{"@@type": "MapView", "controller": True}],
             initial_view_state=view_state,
             map_style = 'dark_no_labels', 
             tooltip=tooltip)
#save
r.to_html("./result/interactiveOnlyManyTrim.html")

**on a laptop without a mouse:**

- `trackpad left-click drag-left` and `-right`;
- `Ctrl left-click drag-up`, `-down`, `-left` and `-right` to rotate and so-on and
- `+` next to Backspace zoom-in and `-` next to `+` zoom-out.

**Now you do your community.** ~ If your area lacks OpenStreetMap data and you want to contribute please follow the [Guide](https://wiki.openstreetmap.org/wiki/Beginners%27_guide).

In [ ]:
## experiments with landuse features

In [297]:
landuse = osm.get_landuse()

In [ ]:
with open('./data/land.geojson', 'w') as outfile:
    json.dump(land_u, outfile)

In [ ]:
data = './data/land.geojson'
json = pd.read_json(data)#data)
land_df = pd.DataFrame()

# Parse the geometry out to Pandas
land_df["coordinates"] = json["features"].apply(lambda row: row["geometry"]["coordinates"])
land_df["type"] = json["features"].apply(lambda row: row["properties"]["osm_landuse"])

color_lookup = pdk.data_utils.assign_random_colors(land_df['type'])
# Assign a color based on attraction_type
land_df['color'] = land_df.apply(lambda row: color_lookup.get(row['type']), axis=1)

In [53]:
land_df.head(2)

In [ ]:
land_u = {
    "type": "FeatureCollection",
    "features": []
    }
columns = landuse.columns   
for i, row in landuse.iterrows():
    f = {
    "type" : "Feature"
    }
    # at a minimum we only want building:levels tagged
    #if row['building:levels'] != 0:
    f["properties"] = {}      
        #-- store all OSM attributes and prefix them with osm_ 
    f["properties"]["osm_id"] = row.id
    #columns = ts.columns
    #for c in columns:
   
    if 'landuse' in columns and row['landuse'] != None:
        f["properties"]["osm_landuse"] = row['landuse']
         
    osm_shape = row["geometry"] # shape(row["geometry"][0])
        #-- a few buildings are not polygons, rather linestrings. This converts them to polygons
        #-- rare, but if not done it breaks the code later
    if osm_shape.type == 'LineString':
        osm_shape = Polygon(osm_shape)
        #-- and multipolygons must be accounted for
    elif osm_shape.type == 'MultiPolygon':
            #osm_shape = Polygon(osm_shape[0])
        for poly in osm_shape:
            osm_shape = Polygon(poly)#[0])
    f["geometry"] = mapping(osm_shape)
    land_u['features'].append(f)
#-- store the data as GeoJSON
